# Convex optimization using CVX

In [50]:
import cvxpy as cp
import numpy as np
import pyscipopt

# Decision variables
x[0] = $F_p$: amplitude of powertrain force (control input) \
x[1] = $\dot{x}$: amplitude of velocity \
x[2] = $a$: binary indicator for whether powertrain max speed is exceeded

In [51]:
Fp = cp.Variable(pos=True)
xdot = cp.Variable(pos=True)
a = cp.Variable(boolean=True)
x = cp.vstack([Fp,xdot,a])

# Objective
Start by trying to maximize mechanical power (not electrical yet) \
max $F_p  \dot{x} = \frac{1}{2} x^T P x $ where $ P = \begin{bmatrix} 0 & 1 & 0 \\ 1 &  0 & 0 \\ 0 & 0 & 0 \end{bmatrix} $ \
This (objective 0) is a quadratic program and it would be ideal to leverage its structure.
P has eigenvalues [1, -1, 0] so it is not positive or negative definite and therefore the QP is not convex or concave. Still, since the objective is a posynomial, convexity/concavity are possible in log-log transformation, so we pursue geometric programming and examine the log-log curvature (objective 1). We also try restricting the domain and optimizing either the inverse or the square root of power (objectives 2 and 3 respectively).

In [52]:
P = np.zeros((3,3))
P[0,1] = 1
P[1,0] = 1
objective_0 = cp.quad_form(x, P)
objective_1 = Fp * xdot
objective_2 = cp.inv_prod(x[0:2]) #objective_2 = cp.inv_pos(cp.inv_prod(x[0:2]))
objective_3 = cp.geo_mean(x[0:2])
print('Objective 0 curvature: ',objective_0.curvature)
print('Objective 1 curvature: ',objective_1.curvature)
print('Objective 2 curvature: ',objective_2.curvature)
print('Objective 3 curvature: ',objective_3.curvature)

objective_1 = cp.Maximize(objective_1) # max power
objective_2 = cp.Minimize(objective_2) # min 1/power
objective_3 = cp.Maximize(objective_3) # max sqrt(power)
print('Objective 1 is log-log convex: ',objective_1.is_dgp())
print('Objective 2 is convex: ',objective_2.is_dcp())
print('Objective 3 is convex: ',objective_3.is_dcp())

Objective 0 curvature:  UNKNOWN
Objective 1 curvature:  LOG-LOG AFFINE
Objective 2 curvature:  CONVEX
Objective 3 curvature:  CONCAVE
Objective 1 is log-log convex:  True
Objective 2 is convex:  True
Objective 3 is convex:  True


# Constraints
Let's try to implement the disjunctive constraint $F_p = 0$ if $\dot{x} \geq x_{max}$, and $F_p \leq F_{max}$ otherwise. This is more realistic to generator capabilities than simply constraining the speed to be within some threshold. (This is motivated separately from convexity, I can use this same disjunctive constraint even if the problem isn't convex). With this constraint I expect a solution of $\dot{x} \approx \dot{x}_{max}$, $F_p = F_{max}$, and $a=0$.
\
\
This can be implemented with four linear constraints using a "big M" formulation and the discrete variable $a \in \{0,1\}$ to control the disjunction: \
$ M a \geq \dot{x} - \dot{x}_{max} $ \
$ M (1 -a) \geq \dot{x}_{max} - \dot{x} $ \
$ F_p \leq F_{max} + a $ \
$ F_p \leq M_2 (1 - a) $

In [53]:
xdotmax = 3
Fmax = 5
M = 1e3
M2 = Fmax + 1

A = np.array([[0, 1, -M], [0, -1, M], [1, 0, -1], [1, 0, M2]])
b = np.array([[xdotmax], [-xdotmax + M], [Fmax], [M2]])

#constraints = [A @ x <= b, xdot <= xdotmax + 10]
constraints = [Fp <= Fmax, xdot <= xdotmax, a <= 0]

is_gdp = [constraint.is_dgp() for constraint in constraints]
print('constraint is GDP: ',is_gdp)
print('Ax-b curvature: ',(A@x-b).curvature)


constraint is GDP:  [True, True, False]
Ax-b curvature:  AFFINE


The constraints are all linear so they make the problems in untransformed coordinates (objectives 2 and 3) convex. However, we check whether it is a valid geometric program constraint, and it is not, so objective 1 won't work.
# Solver

In [54]:
problem = cp.Problem(objective_2, constraints)
print('Constrained problem iS DGP: ',problem.is_dgp())
print('Constrained problem is DCP: ',problem.is_dcp())
print(cp.installed_solvers())
problem.solve(verbose=True)#gp=True)

print("Optimal value: ", problem.value)
print("Fp: ", x[0].value)
print("xdot: ", x[1].value)
print("a: ", x[2].value)

Constrained problem iS DGP:  False
Constrained problem is DCP:  True
(CVXPY) Nov 30 03:20:33 AM: Encountered unexpected exception importing solver OSQP:
ImportError('DLL load failed while importing qdldl: The specified module could not be found.')
['CLARABEL', 'CVXOPT', 'ECOS', 'ECOS_BB', 'GLPK', 'GLPK_MI', 'SCIP', 'SCIPY', 'SCS']
                                     CVXPY                                     
                                     v1.4.1                                    
(CVXPY) Nov 30 03:20:33 AM: Your problem has 3 variables, 3 constraints, and 0 parameters.
(CVXPY) Nov 30 03:20:33 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Nov 30 03:20:33 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Nov 30 03:20:33 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Nov 30 03:20:33 AM: Your problem is compiled with the CPP canoni

-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
(CVXPY) Nov 30 03:20:33 AM: Invoking solver SCIP  to obtain a solution.
-------------------------------------------------------------------------------
                                    Summary                                    
-------------------------------------------------------------------------------
(CVXPY) Nov 30 03:20:33 AM: Problem status: optimal
(CVXPY) Nov 30 03:20:33 AM: Optimal value: 1.721e-02
(CVXPY) Nov 30 03:20:33 AM: Compilation took 4.504e-02 seconds
(CVXPY) Nov 30 03:20:33 AM: Solver (including time spent in interface) took 1.640e-01 seconds
Optimal value:  0.01721325882007964
Fp:  [5.00000005]
xdot:  [3.00000003]
a:  [0.]
